# Data Clean Up

Libraries

In [1]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import datetime
import numpy as np
import xlwings as xw
from pathlib import Path
import fred_functions as ff
import index_prices as ip

### ETF Daily Returns

Import ETF file

In [2]:
df_prices = pd.read_excel('../df_returns.xlsx',index_col=0,sheet_name='Sheet1')

Select "SPY" and "VBMFX". SPY inception was in 1993 while VBMFX was started in 1986. The data set will cover dates from February 1, 1993 to September 30, 2025

In [3]:
df_prices = df_prices[['SPY','VBMFX']] # select tickets
# df = df.loc['1993-02-01':'2025-09-30'] # filter for dates
df_prices.index = pd.to_datetime(df_prices.index)
df_prices = df_prices.sort_index() # sorting
df_prices = df_prices.resample("ME").last().dropna(how='all') # get the month end data, dropnas
df_simple_returns = df_prices.pct_change().dropna(how='all') # find the percent change from the prev month, dropnas
df_cumm_returns = ip.cummulative_return(df_simple_returns,['SPY','VBMFX'])
df_ln_returns = ip.ln_return(df_simple_returns,['SPY','VBMFX'])
df_funds = df_cumm_returns.combine_first(df_ln_returns).sort_index()


## Macro Indicators (FRED)

import macro File

In [4]:
df_macro = pd.read_excel('../df_macro.xlsx',index_col=0,sheet_name='Sheet1')
# df_macro = df_macro.loc['1990-01-01':'2025-09-30'] # filter for dates

### GDP Data Preprocessing

To prepare quarterly GDP data for our model, we perform a two-step process to handle its frequency and reporting lag:

1.  **Resample to Monthly Frequency:** Since GDP is released **quarterly**, we first resample it to a monthly series. This is done using a **forward-fill** (`ffill`) method, where the GDP value for a given quarter is applied to each month within that quarter. For example, the Q1 GDP value is used for January, February, and March.

2.  **Simulate Reporting Lag:** In practice, GDP figures are not known in real-time (e.g., January's final GDP might not be available until April). To simulate this delay and prevent lookahead bias, we **shift** the resampled monthly data forward. We will create four lagged features: a 1-month lag ,2-month lag, 3-month lag, and a 4-month lag.

In [5]:
df_gdp = ff.forward_fill(df_macro[['RealGDP']].copy())
df_gdp = ff.resample_me(df_gdp)
df_gdp = ff.yoy_change(df_gdp)
# df_gdp_lag1 = ff.shift_lag(df_gdp,lag_months=1).rename(columns={"RealGDP":"RealGDP_1m"})
# df_gdp_lag2 = ff.shift_lag(df_gdp,lag_months=2).rename(columns={"RealGDP":"RealGDP_2m"})
df_gdp_lag3 = ff.shift_lag(df_gdp,lag_months=3).rename(columns={"RealGDP":"RealGDP_3m"})
# df_gdp_lag4 = ff.shift_lag(df_gdp,lag_months=4).rename(columns={"RealGDP":"RealGDP_4m"})
# df_gdp_clean = df_gdp.join([df_gdp_lag1, df_gdp_lag2,df_gdp_lag3,df_gdp_lag4], how='inner') # use this to join the lag_df
df_gdp_clean = df_gdp.join([df_gdp_lag3], how='inner') # only using lag3

### Permits, CPI, CoreCPI, IndustrialProd, RealRetail


In [6]:
indicator_index_cols = ['Permits','CPI','CoreCPI','IndustrialProd','RealRetail'] # select columns to update
df_indicator_index = df_macro[indicator_index_cols].copy() # make a copy
df_indicator_index = ff.resample_me(df_indicator_index) # get month-end indices
df_lag1_indicator_index = ff.shift_lag(df_indicator_index,lag_months=1) #shift by 1 to account for information lag
# dictionary to rename original numbers to "_lag1"
col_rename_indicator_index = {
    'Permits':'Permits_1m',
    'CPI':'CPI_1m',
    'CoreCPI':'CoreCPI_1m',
    'IndustrialProd':'IndustrialProd_1m',
    'RealRetail':'RealRetail_1m'
}
col_rename_yoy = {
    'Permits_1m':'Permits_1m_yoy',
    'CPI_1m':'CPI_1m_yoy',
    'CoreCPI_1m':'CoreCPI_1m_yoy',
    'IndustrialProd_1m':'IndustrialProd_1m_yoy',
    'RealRetail_1m':'RealRetail_1m_yoy'
}
ff.rename_columns(df_lag1_indicator_index,col_rename_indicator_index) #ff to rename columns using the dict above
df_indicator_yoy = ff.yoy_change(df_lag1_indicator_index) # ff for year over year change
ff.rename_columns(df_indicator_yoy,col_rename_yoy) # rename yoy columns
df_indicator_mom = ff.mom_change(df_lag1_indicator_index) ## ff for month-over-month change
df_indicator_mom = ff.rolling_3m(df_indicator_mom,df_indicator_mom.columns) # ff to calculate 3month rolling avg, and also annualized 3month rolling avg
df_indicator_clean = df_indicator_index.join([df_indicator_yoy,df_indicator_mom], how='inner')

### Unemployment

In [7]:
df_unemployment = df_macro[["UnemploymentRate"]].copy()
df_unemployment = ff.forward_fill(df_unemployment)
df_unemployment = ff.resample_me(df_unemployment)
df_unemployment_1m = ff.shift_lag(df_unemployment,lag_months=1)
df_unemployment_1m = ff.mom_change(df_unemployment_1m)
df_unemployment_clean = ff.rolling_3m(df_unemployment_1m,df_unemployment_1m.columns)

### 

### Rates - Mortgage 30 Year, US 10 YR Treasury, US 2 Year Treasury, US 3 Month Treasury, TIPS10 YR, CorpBAA, CorpAAA, FedFunsRate


In [8]:
df_rate_clean = ff.credit_spread(df_macro,lag_months=0,delta_calc=True)
col_remove = ['Mort30Y', 'UST10Y', 'UST2Y', 'UST3M','TIPS10Y', 'CorpBAA', 'CorpAAA','FedFundsRate']
df_rate_clean.drop(columns=col_remove,inplace=True)

In [9]:
df_macro_clean = (
    df_gdp_clean.combine_first(df_indicator_clean)
       .combine_first(df_unemployment_clean)
       .combine_first(df_rate_clean)
       .sort_index()
)

In [10]:
df_macro_clean

,AAA_minus_10Y,AAA_minus_10Y_Delta1m,AAA_minus_10Y_Delta3m,BAA_minus_10Y,BAA_minus_10Y_Delta1m,BAA_minus_10Y_Delta3m,BAA_minus_AAA,BAA_minus_AAA_Delta1m,BAA_minus_AAA_Delta3m,CPI,...,TIPS10Y_Delta1m,TIPS10Y_Delta3m,UST10Y_Delta1m,UST10Y_Delta3m,UST2Y_Delta1m,UST2Y_Delta3m,UST3M_Delta1m,UST3M_Delta3m,UnemploymentRate_ann_r3m,UnemploymentRate_r3m
DATE,,,,,,,,,,,,,,,,,,,,,
1985-01-31,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-02-28,0.65,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.300,...,NaN,NaN,0.74,NaN,0.76,NaN,0.48,NaN,NaN,NaN
1985-03-31,0.77,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.800,...,NaN,NaN,-0.26,NaN,-0.23,NaN,-0.34,NaN,NaN,NaN
1985-04-30,0.77,0.00,0.12,NaN,NaN,NaN,NaN,NaN,NaN,107.000,...,NaN,NaN,-0.24,0.24,-0.52,0.01,-0.35,-0.21,NaN,NaN
1985-05-31,0.89,0.12,0.24,NaN,NaN,NaN,NaN,NaN,NaN,107.200,...,NaN,NaN,-1.13,-1.63,-0.99,-1.74,-0.74,-1.43,0.000761,0.000063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,1.10,0.00,0.02,1.75,-0.09,-0.01,0.65,-0.09,-0.03,321.500,...,-0.12,0.10,-0.17,0.01,-0.17,-0.17,0.05,0.09,0.102044,0.008130
2025-07-31,1.04,-0.06,-0.17,1.67,-0.08,-0.29,0.63,-0.02,-0.12,322.132,...,0.03,0.04,0.13,0.20,0.22,0.34,0.00,0.10,-0.091189,-0.007937
2025-08-31,1.19,0.15,0.09,1.80,0.13,-0.04,0.61,-0.02,-0.13,323.364,...,-0.16,-0.25,-0.14,-0.18,-0.35,-0.30,-0.18,-0.13,0.002325,0.000194
